In [1]:
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
import cv2
import math
import utils


2021-07-27 00:16:56.266928: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [4]:
for i in range(0, 121):
    if (i % 2 == 0): continue
    filename = f"/home/dan/Downloads/poulespng/poule-{i}.pdf.png"
    print(filename)
    orig = cv2.imread(filename, cv2.IMREAD_UNCHANGED)

    try:
        utils.straighten_front(orig, debug=True, pad=200)
    except:
        print

IndentationError: expected an indented block (3277699023.py, line 10)